In [32]:
# Allow multiple print statements in a cell in Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [21]:
import numpy as np
import csv

In [22]:
DATA_PATH = "./data"
DATASET_FILES = {
    "ITALIAN": {
        "TRAIN": f"{DATA_PATH}/it_isdt_train_tagged.txt",
        "DEV": {
            "RAW": f"{DATA_PATH}/it_isdt_dev_raw.txt",
            "TAGGED": f"{DATA_PATH}/it_isdt_train_tagged.txt",
        },
    },
    "JAPANESE": {
        "TRAIN": f"{DATA_PATH}/ja_gsd_train_tagged.txt",
        "DEV_RAW": f"{DATA_PATH}/ja_gsd_dev_raw.txt",
        "DEV_TAGGED": f"{DATA_PATH}/ja_gsd_train_tagged.txt",
    },
}
MODEL_FILE = "hmmmodel.txt"


In [ ]:
START_TAG = "<START>"
END_TAG = "<END>"

In [33]:
document = list()

In [34]:
with open(DATASET_FILES["ITALIAN"]["TRAIN"], mode="r") as file:
    csv_reader = csv.reader(file, delimiter=" ", skipinitialspace=True, quoting=csv.QUOTE_NONE)
    for sentence in csv_reader:
        document.append(sentence)

In [35]:
tag_count_dict = {}
word_count_dict = {}

In [36]:
count = len(document)
# Process count number of sentences from document
for idx, sentence in enumerate(document):
    if idx == count:
        break
    for word_tag_pair in sentence:
        # Make the word and tag lower and extract word tag
        word, tag = word_tag_pair.lower().rsplit("/", 1)

        # Count the Tag!
        if tag not in tag_count_dict:
            tag_count_dict[tag] = 1
        else:
            tag_count_dict[tag] += 1

        # Count the Word!
        if word not in word_count_dict:
            word_count_dict[word] = {tag: 1}
        else:
            # Check if the tag is in the dict
            if tag not in word_count_dict[word]:
                word_count_dict[word][tag] = 1
            else:
                word_count_dict[word][tag] += 1


In [37]:
tag_count_dict, word_count_dict

({'sp': 13670,
  'fs': 11925,
  's': 54975,
  'e': 41893,
  'rd': 35237,
  'a': 17349,
  'fc': 1241,
  'ri': 4284,
  'va': 5923,
  'vm': 1758,
  'v': 26560,
  'pr': 2879,
  'fb': 5436,
  'pi': 949,
  'ff': 12692,
  'cc': 7549,
  'n': 4795,
  'pc': 4333,
  'ap': 1701,
  'no': 976,
  'b': 8701,
  'i': 62,
  't': 379,
  'pe': 647,
  'cs': 2833,
  'di': 1502,
  'bn': 1827,
  'pd': 770,
  'dd': 1014,
  'de': 4,
  'dq': 875,
  'pq': 823,
  'sw': 214,
  'pp': 29,
  'dr': 39,
  'sym': 88,
  'x': 63,
  'part': 24},
 {'londra': {'sp': 31},
  '.': {'fs': 9738, 'ff': 207},
  'gas': {'s': 26},
  'da': {'e': 2946},
  'la': {'rd': 8833, 'pc': 128, 'sp': 2, 'sw': 4, 's': 1},
  'statua': {'s': 5, 'sp': 2},
  'inconsueto': {'a': 1},
  'allarme': {'s': 15},
  'a': {'e': 6686, 'sp': 20, 's': 9, 'b': 2, 'x': 8, 'ri': 3, 'rd': 1},
  'tate': {'sp': 1},
  'gallery': {'sp': 1, 's': 1},
  ':': {'fc': 794, 'fs': 1, 'ff': 1},
  'una': {'ri': 1501, 'pi': 63, 'n': 17, 's': 1, 'a': 1},
  'sala': {'s': 15, 'sp': 2},


In [ ]:
# TODO: Create Emission Probability
# TODO: Create Transisiton Probability